<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/T5_GrammarFixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install sentencepiece
!pip install transformers
!pip install datasets

In [2]:
# Basics
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# PyTorch
import torch 
from torch.utils.data import DataLoader

# Sklearn
from sklearn.model_selection import train_test_split

# Transformers
from transformers import logging
logging.set_verbosity_error()

from transformers import T5TokenizerFast, T5PreTrainedModel, T5ForConditionalGeneration, AdamW

# Datasets
import datasets

In [3]:
# Dummy Dataset
df = pd.read_parquet('/content/drive/MyDrive/Dirty-Talks/Topic Modeling/data/dbpedia_translated_pandas.parquet').sample(n=1000)
df['true'] = df.doc
df['pred'] = df.doc
df = df[['true', 'pred']]

# Dummy split
X_train, X_test, y_train, y_test = train_test_split(df.pred, df.true, test_size=0.10, random_state=42)
df_train = pd.concat((X_train, y_train), axis=1).reset_index(drop=True)
df_test = pd.concat((X_test, y_test), axis=1).reset_index(drop=True)

train_pred_idx = {v:k for k, v in enumerate(df_train.pred.unique())}
df_train = df_train.assign(pred_idx = df_train.pred.apply(lambda x: train_pred_idx[x]))

train_true_idx = {v:k for k, v in enumerate(df_train.true.unique())}
df_train = df_train.assign(true_idx = df_train.true.apply(lambda x: train_true_idx[x]))

test_pred_idx = {v:k for k, v in enumerate(df_test.pred.unique())}
df_test = df_test.assign(pred_idx = df_test.pred.apply(lambda x: test_pred_idx[x]))

test_true_idx = {v:k for k, v in enumerate(df_test.true.unique())}
df_test = df_test.assign(true_idx = df_test.true.apply(lambda x: test_true_idx[x]))


df_train

,pred,true,pred_idx,true_idx
0,O Grande Prémio de Futsal de 2010 foi a sexta ...,O Grande Prémio de Futsal de 2010 foi a sexta ...,0,0
1,"My Hero (Seishun Scandal, Japão) é um videogam...","My Hero (Seishun Scandal, Japão) é um videogam...",1,1
2,"Ings é uma palavra antiga de origem nórdica, q...","Ings é uma palavra antiga de origem nórdica, q...",2,2
3,Pheax (em grego: Φαία) foi um orador e estadis...,Pheax (em grego: Φαία) foi um orador e estadis...,3,3
4,O Sindicato Geral dos Trabalhadores (Unión Gen...,O Sindicato Geral dos Trabalhadores (Unión Gen...,4,4
...,...,...,...,...
895,Durbania limbata é uma borboleta da família Ly...,Durbania limbata é uma borboleta da família Ly...,895,895
896,John H. Dialogue foi um industrial nascido na ...,John H. Dialogue foi um industrial nascido na ...,896,896
897,Paul Tiffany é professor senior na Faculdade d...,Paul Tiffany é professor senior na Faculdade d...,897,897
898,"Richard Estes (Kewanee, Illinois, 14 de maio d...","Richard Estes (Kewanee, Illinois, 14 de maio d...",898,898


In [4]:
df_test

,pred,true,pred_idx,true_idx
0,"Em enzimologia, uma biocanina-A reductase (EC ...","Em enzimologia, uma biocanina-A reductase (EC ...",0,0
1,Arctostaphylos pechoensis é uma espécie de man...,Arctostaphylos pechoensis é uma espécie de man...,1,1
2,"Barclay House, também conhecida como Joshua Ha...","Barclay House, também conhecida como Joshua Ha...",2,2
3,"Erkka Petäjä (Madrid, 13 de fevereiro de 1964)...","Erkka Petäjä (Madrid, 13 de fevereiro de 1964)...",3,3
4,"Sean Brewer (Sr. Sean, 5 de outubro de 1977) é...","Sean Brewer (Sr. Sean, 5 de outubro de 1977) é...",4,4
...,...,...,...,...
95,"Pascual Luna Parra (Pascual, 22 de fevereiro d...","Pascual Luna Parra (Pascual, 22 de fevereiro d...",95,95
96,Aliança pela Justiça (AFJ) é um grupo progress...,Aliança pela Justiça (AFJ) é um grupo progress...,96,96
97,"O Troféu Herder Memorial, ou Herder, é o trofé...","O Troféu Herder Memorial, ou Herder, é o trofé...",97,97
98,Fotbal Club Universitatea Cluj () é um clube d...,Fotbal Club Universitatea Cluj () é um clube d...,98,98


In [5]:
# To dataset
df_train = datasets.Dataset.from_pandas(df_train)
df_test = datasets.Dataset.from_pandas(df_test)

df_train

Dataset({
    features: ['pred', 'true', 'pred_idx', 'true_idx'],
    num_rows: 900
})

In [6]:
df_test

Dataset({
    features: ['pred', 'true', 'pred_idx', 'true_idx'],
    num_rows: 100
})

In [7]:
# Tokenize train
tokenizer = T5TokenizerFast.from_pretrained('t5-small')
MAX_LEN = 64

data_train = df_train.map(
    lambda x: tokenizer(
            x['true'], 
            max_length=MAX_LEN, 
            padding='max_length',
            truncation=True), 
            batched=True)

data_train = data_train.rename_column('input_ids', 'true_ids')
data_train = data_train.rename_column('attention_mask', 'true_mask')

data_train = data_train.map(
    lambda x: tokenizer(
            x['pred'], 
            max_length=MAX_LEN, 
            padding='max_length',
            truncation=True), 
            batched=True)

data_train = data_train.rename_column('input_ids', 'pred_ids')
data_train = data_train.rename_column('attention_mask', 'pred_mask')

data_train = data_train.remove_columns(['true', 'pred', 'true_mask'])

data_train

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['pred_mask', 'pred_ids', 'pred_idx', 'true_ids', 'true_idx'],
    num_rows: 900
})

In [8]:
# Tokenize test

data_test = df_test.map(
    lambda x: tokenizer(
            x['true'], 
            max_length=MAX_LEN, 
            padding='max_length',
            truncation=True), 
            batched=True)

data_test = data_test.rename_column('input_ids', 'true_ids')
data_test = data_test.rename_column('attention_mask', 'true_mask')

data_test = data_test.map(
    lambda x: tokenizer(
            x['pred'], 
            max_length=MAX_LEN, 
            padding='max_length',
            truncation=True), 
            batched=True)

data_test = data_test.rename_column('input_ids', 'pred_ids')
data_test = data_test.rename_column('attention_mask', 'pred_mask')

data_test = data_test.remove_columns(['true', 'pred', 'true_mask'])

data_test

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['pred_mask', 'pred_ids', 'pred_idx', 'true_ids', 'true_idx'],
    num_rows: 100
})

In [9]:
data_train.set_format(type='torch', output_all_columns=True)
data_test.set_format(type='torch', output_all_columns=True)

train_loader = torch.utils.data.DataLoader(
    data_train, 
    batch_size=4, 
    shuffle=True,
    pin_memory=False,
    num_workers=2,
    )

test_loader = torch.utils.data.DataLoader(
    data_test, 
    batch_size=4, 
    shuffle=False,
    pin_memory=False,
    num_workers=2,
    )


dl_sizes = {i: len(x) for i, x in enumerate((train_loader, test_loader))}
print(dl_sizes)

dl0 = next(iter(test_loader))
dl0.keys()

{0: 225, 1: 25}


dict_keys(['pred_mask', 'pred_ids', 'pred_idx', 'true_ids', 'true_idx'])

In [10]:
model = T5ForConditionalGeneration.from_pretrained("t5-small", return_dict=True)

outputs_fwd = model(input_ids=dl0['pred_ids'], attention_mask=dl0['pred_mask'], labels=dl0['true_ids'])
print(outputs_fwd.keys())

outputs = model.generate(dl0['pred_ids'])
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])
         


In [11]:
# acc metric for text inputs
def acc_in_text(trues, preds): 
    acc = []
    for d in zip(trues, preds):
        if d[0] == d[1]:
            acc.append(1)
        else:
            acc.append(0)
    return acc # bool

def train(epoch, model, loader, optim, device='cuda'):
    model.train()  
    loop = tqdm(loader, leave=True)
    loss_train = []
    for batch in loop:
        model.zero_grad()

        source_ids, source_mask, target_token_ids = (
            b.to(device) for b in (batch['pred_ids'], batch['pred_mask'], batch['true_ids'])
            )

        outs = model(
            input_ids=source_ids, 
            attention_mask=source_mask, 
            labels=target_token_ids,
            )
        
        loss_train.append(outs['loss'].item())
        outs['loss'].backward()
        optim.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=outs['loss'].item())

    train_losses = sum(loss_train) / len(loss_train)

@torch.no_grad()  
def evaluate(model, loader, device='cuda', max_len=64):
    all_acc, all_preds, all_trues = [], [], []
    model.eval()

    loop = tqdm(loader, leave=True)

    for batch in loop:
        model.zero_grad()

        source_ids, source_mask, target_token_ids = (
            b.to(device) for b in (batch['pred_ids'], batch['pred_mask'], batch['true_ids'])
            )
        predicted_ids = model.generate(
            source_ids.to(device), 
            max_len
            )
        preds = [tokenizer.decode(t, skip_special_tokens=True) for t in predicted_ids]
        trues = [tokenizer.decode(t, skip_special_tokens=True) for t in target_token_ids]
        acc = acc_in_text(trues, preds)
        all_acc.extend(acc)
        all_trues.extend(trues)
        all_preds.extend(preds)

        loop.set_description('Evaluating')
        loop.set_postfix(acc=np.array(all_acc).mean())
        
    
    return all_trues, all_preds

# Training 

In [12]:
model.to('cuda')
optimizer = AdamW(model.parameters(), lr=5e-5)

WINDOW = 3
# ---------------------------------------------------------------------------------
N_EPOCHS = 2
for step in range(1, N_EPOCHS+1):
    samp = random.randint(0, 4) # to show random trues and preds
    train(step, model, train_loader, optimizer,)
    trues, preds = evaluate(model, test_loader)
    print(f'  Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  Trues: ['Erkka Petäjä (Madrid, 13 de fevereiro de 1964) é um ex-futebolista finlandês que atuava como jogador de futebol internacional. Petäjä, que atuou', 'Sean Brewer (Sr. Sean, 5 de outubro de 1977) é um ex-futebolista americano. Ele foi recrutado pelos Bengales Cincinnati na terceira rodada do Draft da NFL 2001. Ele jogo', 'A rodovia 22 em Haryana é para Bahadurgarh-Jhajjar-Kosli. É sem divisores e só nas cidades há divisor. No há um']
  Preds: ['Erkka Petäjä (Madrid, 13 de fevereiro de 1964) é um ex-futebolista finlandês que atuava como jogador de futebol internacional. Petäjä, que atuou', 'Sean Brewer (Sr. Sean, 5 de outubro de 1977) é um ex-futebolista americano. Ele foi recrutado pelos Bengales Cincinnati na terceira rodada do Draft da NFL 2001. Ele jogo', 'A rodovia 22 em Haryana é para Bahadurgarh-Jhajjar-Kosli. É sem divisores e só nas cidades há divisor. No há um']


  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  Trues: ['Erkka Petäjä (Madrid, 13 de fevereiro de 1964) é um ex-futebolista finlandês que atuava como jogador de futebol internacional. Petäjä, que atuou', 'Sean Brewer (Sr. Sean, 5 de outubro de 1977) é um ex-futebolista americano. Ele foi recrutado pelos Bengales Cincinnati na terceira rodada do Draft da NFL 2001. Ele jogo', 'A rodovia 22 em Haryana é para Bahadurgarh-Jhajjar-Kosli. É sem divisores e só nas cidades há divisor. No há um']
  Preds: ['Erkka Petäjä (Madrid, 13 de fevereiro de 1964) é um ex-futebolista finlandês que atuava como jogador de futebol internacional. Petäjä, que atuou', 'Sean Brewer (Sr. Sean, 5 de outubro de 1977) é um ex-futebolista americano. Ele foi recrutado pelos Bengales Cincinnati na terceira rodada do Draft da NFL 2001. Ele jogo', 'A rodovia 22 em Haryana é para Bahadurgarh-Jhajjar-Kosli. É sem divisores e só nas cidades há divisor. No há um']
